In [12]:
## plot
%matplotlib inline
import matplotlib.pyplot as plt

## operations
import numpy as np
from scipy.io import loadmat
from scipy import signal
from scipy.signal import butter, lfilter
from scipy import stats 

## time_frequency
from mne.time_frequency import tfr_array_multitaper, psd_array_multitaper

## pac
from pactools import Comodulogram, REFERENCES
from pactools import simulate_pac

import pandas as pd

from statsmodels.formula.api import ols, mixedlm
import statsmodels.api as sm


## Custom made functions from 'util_functions.py' script
from util_functions import *

## Saving data dictionaries
import pickle  ``

# Data files info

In [6]:
sessions = ['Eugeni_s2','Eugeni_s3','JCQ_s1','JCQ_s2','JS_s1','JS_s2','JS_s3','MRI_s2']  ## missing 'MM_s1'.. ttl needs to be fixed
ttl_idx  = [123, 123, 125, 125, 120,120, 120, 126 ]

In [7]:
chan_of_int = [ "B'1" , "B'2"  , "C'1", "C'2", "B1", "B2", "C1", "C2"  ]

hpc_electrodes = [] ## make a list of hippocampal electrodes index 
for pp in range(len(sessions)):

    ss = sessions[pp]
    chan_info = loadmat('../Data/'+ss+'_chanlocs_labels.mat')
    indx_chan = []

    for ii in range( chan_info['x'].shape[0] ):
        if chan_info['x'][ii][0][0] in chan_of_int: indx_chan.append( ii)
    hpc_electrodes.append( indx_chan )

In [8]:
trials_markers = {}

to_plot = False ## a flag to plot trials index

for n in range(len(sessions)):

    ss = sessions[n]
    ttl = ttl_idx[n]

    lfp = loadmat('../Data/'+ss+'.mat')

    ttl_diff = np.diff( lfp['ourData'][ttl] )

    ttl_diff_xx = np.zeros_like(ttl_diff)
    ttl_diff_xx[ ttl_diff > 100  ] = 350

    idx = np.where(ttl_diff_xx!=0)[0]
    idx = idx[ np.where(np.diff(idx)>500)[0] ]

    last_trial = np.where(ttl_diff_xx!=0)[0][-1]

    idx_trials = np.concatenate(np.array([ idx , np.array([last_trial])     ]) )

    if to_plot == True:
        plt.figure(figsize=(15,3))
        plt.plot(  ttl_diff )
        plt.title(ss)
        plt.vlines(idx_trials, 0,400, 'g')

    trials_markers[ ss ] = idx_trials

In [14]:
### Save trial markers in a pickle file
with open('../Results/trials_markers.pickle', 'wb') as handle:
    pickle.dump(trials_markers, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    
### Load trial markers for each session
# with open('../Results/trials_markers.pickle', 'rb') as handle:
#     trials_markers = pickle.load(handle)

In [19]:
fs = 512. ## Sampling rate (Hz)

## Trial type related info
density = np.tile( np.repeat( np.arange(3), 3 ),3)
speed = np.tile( np.arange(3), 9 )

# dur = [ 18.1, 13.5, 11]
# dd = [50,100,150]
dur = np.tile( np.array([ 18.1, 13.5, 11]), 3).reshape(3,3).T
dd = np.tile(np.array([50,100,150]),3).reshape(3,3)
K_i = dd/dur
K_i_idx = np.unravel_index(np.argsort(K_i, axis=None), K_i.shape)
uni = np.unique(K_i)

t_dur = 360/np.array([ 18.1, 13.5, 11])
t_dd = np.array([50,100,150])


study_info = {}
study_info['density'] = density
study_info['speed'] = speed
study_info['dur'] = dur
study_info['dd'] = dd
study_info['fs'] = fs
study_info['K_i_idx'] = K_i_idx
study_info['K_i'] = K_i
study_info['uni'] = uni
study_info['t_dur'] = t_dur
study_info['t_dd'] = t_dd


study_info['sessions'] = sessions
study_info['ttl_idx'] = ttl_idx 

study_info['hpc_electrodes'] = hpc_electrodes


### Save trial markers in a pickle file
with open('../Results/study_info.pickle', 'wb') as handle:
    pickle.dump(study_info, handle, protocol=pickle.HIGHEST_PROTOCOL)
